In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **Preprocessing**  
I decided to make a customized preprocessing Function to be used in different problems.  




By customized I mean you can choose the cleaning functions you want applied on your text or even choose between 2 different languages to clean (Arabic or English).  

The function dafault Arguments:

* lang = 'en' :**Works on English Language, 'ar' for Arabic** 
* rm_extras = True :**Remove punctuations, numbers and special characters**
* lower = True :**Change English texts to lowercase**
* stopwords = True :**Remove Stopwords**
* stemming = True :**Stem texts**
* rm_contraction = True :**Maps contracted words to it's original**
* rm_tashkel = True :**Remove Tashkel From arabic texts**
* rm_short = False :**Remove Tashkel From arabic texts**

you can change any of the True above arguments to false to prevent that particular cleaning.


---

## **Downloading libraries And importing stopwords**

The following code run once to download needed libraries and read the customized arabic stop words.
Make sure to change directory of text file of stop words to your downloaded path.

The arabic stop words have been obtained from https://github.com/mohataher/arabic-stop-words

In [0]:
!pip install PyArabic
!pip install tashaphyne
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Read arabic stopwords text file.
filepath = "/content/drive/My Drive/Colab Notebooks/Text_Summarization_Task/Word_Embedding/Arabic_Stopwords.txt"
f = open(filepath, 'r')
stop_word_comp = [line.rstrip('\n') for line in f.readlines()]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## **Import the Libraries**

In [0]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pyarabic.araby as araby
from tashaphyne.stemming import ArabicLightStemmer
from textblob import TextBlob
import re

## Functions Used in the main preprocess function separated to reduce the complexity of the function




In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

    
def arabic_stem(text):
    ArListem = ArabicLightStemmer()
    zen = TextBlob(text)
    words = zen.words
    cleaned = list()
    for w in words:
        ArListem.light_stem(w)
        cleaned.append(ArListem.get_root())
    return " ".join(cleaned)

def normalizeArabic(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    return araby.strip_tashkeel(text)


def remove_stop_words(text , lang ):
    if (lang == 'en'):
       stop_words = stopwords.words('english')
    else:
       stop_words = stopwords.words('arabic')
    
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stop_words and not w in stop_word_comp and len(w) >= 2])

def remove_short_words(text):
  not_so_short_text = ''
  for t in text.split(' '):
    if len(t) >= 3:
      not_so_short_text = not_so_short_text + ' ' + t
  return not_so_short_text


## Main Preprocessing Function

In [0]:
def preprocess(text, lang="en" , rm_extras = True , lower = True , stopwords = True , stemming = True , rm_tashkel = True  , rm_contraction = True , rm_short = False):
    if (lang == "en"):
      
      # make alphabets lowercase
      if(lower):
        text  = text.lower()

      #remove stopwords
      if(stopwords):
        text = remove_stop_words(text , 'en')

      # Expand contrated words
      if (rm_contraction):
        text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])

      # remove punctuations, numbers and special characters
      if(rm_extras):
        text  = re.sub("[^a-zA-Z]", " " , text)

      # Stemming texts
      if(stemming):
        porter = PorterStemmer()
        text = porter.stem(text)
      
      # Removing short texts
      if(rm_short):
        text = remove_short_words(text)

      #remove spaces from endlines
      text = text.strip()

    if (lang=="ar"):
      # remove punctuations, numbers and special characters
      if(rm_extras):
        text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
        ## remove extra whitespace
        text = re.sub('\s+', ' ', text)
        ## Remove numbers
        text = re.sub("\d+", " ", text)
      
      ## Remove stop words
      if(stopwords):
        text = remove_stop_words(text , 'ar')
      
      if(rm_tashkel):
        ## Remove Tashkeel
        text = normalizeArabic(text)
        text = re.sub('[A-Za-z]+',' ',text)
        text = re.sub(r'\\u[A-Za-z0-9\\]+',' ',text)

      ## Remove extra whitespace
      text = re.sub('\s+', ' ', text)  

      ## Stemming
      if(stemming):
        text = arabic_stem(text)

      #remove spaces from endlines
      text = text.strip()
      
    return text

# **Reading From wikipedia**
This function is simply used to read a wikipedia url, Clean it and return list of sentences.



we begin by importing the essential libraries for fetching data from the web page. The BeautifulSoup library is used for parsing the page while the urllib library is used for connecting to the page and retrieving the HTML.

BeautifulSoup converts the incoming text to Unicode characters and the outgoing text to UTF-8 characters, saving you the hassle of managing different charset encodings while scraping text from the web.

We’ll use the urlopen function from the urllib.request utility to open the web page. Then, we’ll use the read function to read the scraped data object. For parsing the data, we’ll call the BeautifulSoup object and pass two parameters to it; that is, the article_read and the html.parser.

The find_all function is used to return all the ``` <p> ``` elements present in the HTML. Furthermore, using .text enables us to select only the texts found within the ``` <p> ``` elements.

In [0]:
import bs4 as BeautifulSoup
import urllib.request

def read_wiki(url):
  #fetching the content from the URL
  fetched_data = urllib.request.urlopen(url)
  article_read = fetched_data.read()

  #parsing the URL content and storing in a variable
  article_parsed = BeautifulSoup.BeautifulSoup(article_read,'html.parser')

  #returning <p> tags
  paragraphs = article_parsed.find_all('p')

  article_content = ''
  
  #looping through the paragraphs and adding them to the variable
  for p in paragraphs:  
      article_content += p.text
  return article_content